In [ ]:
from sklearn.metrics import confusion_matrix
from numpy import ndarray
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
from __future__ import print_function
import numpy as np
import sklearn

In [ ]:
# Extracting the data set
mnist = input_data.read_data_sets("/data/", one_hot=True)

In [ ]:
# parameters intialized 
learning_rate = 0.0001
training_epochs = 5
batch_size = 100
display_step = 0.5
a = np.zeros(shape=(10,10))

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [ ]:
# Graph of Tensor Flow
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

In [ ]:
# Weights and Bias 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# RBF - Radial Basis 
def RBF(x, C):
    """Computes distance from cluster centers defined in input C
    
    Both outdim and indim should be integers.
    """
    return -tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(tf.expand_dims(x,2),
                                                   tf.expand_dims(C,0))),1))

In [ ]:
# Creating MLP model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    return layer_1


In [ ]:
# Constructing a model
input_X = multilayer_perceptron(X)

logits = RBF(input_X,weights['out'])+biases['out']


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
i = 0
train_accuracy = np.zeros((1,100))
avg_cost = np.zeros((1,100))
count = np.zeros((1,100))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(10000):
        batch = mnist.train.next_batch(100)
        

        if i%100 == 0:
            j = int(i/100)
            print(j)
            count[0][j-1] = j
            train_accuracy[0][j-1] = accuracy.eval(feed_dict={
                X:batch[0], Y: batch[1]})
            _,c = sess.run([train_step,cross_entropy],feed_dict={X:batch[0], Y:batch[1]})
            total_batches = j
            avg_cost[0][j-1] = c/total_batches 
            print("\rstep %d, training accuracy %g"%(i, train_accuracy[0][j-1]), end="" if i%10 else "\n")
        train_step.run(feed_dict={X: batch[0], Y: batch[1]})
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(" Testing Accuracy:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    pred = tf.nn.softmax(logits)  # Applyinf Softmax to logits
    confusionmatrix = tf.contrib.metrics.confusion_matrix(tf.argmax(Y,1), tf.argmax(pred,1))
    print('confusion matrix \n', tf.Tensor.eval(confusionmatrix,feed_dict={X: mnist.train.images, Y: mnist.train.labels}, session= None) )


In [ ]:
X = ([0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 7900])
Y = ([0.04, 0.64, 0.78, 0.88, 0.89, 0.89, 0.87, 0.91, 0.9209])
plt.plot(X,Y,'b--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy as a function of Epoch in RBF')
plt.legend(loc = "best")
plt.show()
